In [ ]:
#@title sshd & drive
#from google.colab import drive
#drive.mount('/content/drive')

#Setup sshd
!apt-get install -qq -o=Dpkg::Use-Pty=0 openssh-server pwgen > /dev/null
!mkdir -p /var/run/sshd
!echo "PermitRootLogin without-password" >> /etc/ssh/sshd_config
!mkdir -p /root/.ssh
!cat gcloud.pub >> /root/.ssh/authorized_keys

#Run sshd
get_ipython().system_raw('/usr/sbin/sshd -D &')

!chmod 400 ./gcloud.pem 
!apt install bc  # used for math

#! echo "LD_LIBRARY_PATH=/usr/lib64-nvidia" >> /root/.bashrc
#! echo "export LD_LIBRARY_PATH" >> /root/.bashrc

In [ ]:
#@title remote tunnel
public_server = "34.70.114.171" #@param {type:"string"}

!chmod 400 gcloud.pem

!ssh -o StrictHostKeyChecking=no -i /content/gcloud.pem -N -vv \
    -R 3022:localhost:22 -R 39152:localhost:49152 \
    daniele.veri.96@$public_server

In [ ]:
#@title ngrok (unused)

#Download ngrok
! wget -q -c -nc https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip -qq -n ngrok-stable-linux-amd64.zip

#Create ngrok tunnel
get_ipython().system_raw(f'./ngrok start -config ./drive/MyDrive/ngrok-config.yaml ssh nvidia-prof &')
#Get public address
! curl -s http://localhost:4040/api/tunnels

In [ ]:
#@title strong scaling speedup plots
import matplotlib.pyplot as plt
import seaborn as sns

x=range(1,17)
yamd = [
7.772,
3.428,
2.974,
1.990,
1.710,
1.736,
1.568,
1.452,
1.442,
1.471,
1.339,
1.392,
1.331,
1.356,
1.303,
1.250]

yint= [
6.012,
3.164,
2.203,
1.722,
1.439,
1.251,
1.114,
1.023,
1.211,
1.142,
1.054,
1.009,
.951,
.902,
.876,
.834]

amdahl = [
1.000,
1.912,
2.747,
3.521,
4.237,
4.878,
5.494,
6.060,
6.578,
7.092,
7.575,
8.000,
8.403,
8.771,
9.174,
9.523]

strong_eff_amd = []
strong_eff_int = []
for i,_ in enumerate(yamd):
    strong_eff_amd.append(yamd[0]/yamd[i])
    strong_eff_int.append(yint[0]/yint[i])

sns.set_style("darkgrid")
plt.figure(figsize=(20,12))
plt.xlabel('thread', fontsize=18)
plt.ylabel('S(p)', fontsize=18)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.ylim(ymin=0, ymax=14)
plt.scatter(x=x, y=strong_eff_amd, s=200)
plt.scatter(x=x, y=strong_eff_int, s=200)
plt.scatter(x=x, y=amdahl, s=200, marker='*')
plt.legend(['AMD EPYC 7B12','Intel Xenon E5-2697','Amdahl law'])

In [ ]:
#@title strong scaling efficiency plots

strong_eff_amd = []
strong_eff_int = []
for i,_ in enumerate(yamd):
    strong_eff_amd.append(yamd[0]/yamd[i]/(i+1))
    strong_eff_int.append(yint[0]/yint[i]/(i+1))

sns.set_style("darkgrid")
plt.figure(figsize=(20,12))
plt.xlabel('thread', fontsize=18)
plt.ylabel('E(p)', fontsize=18)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.ylim(ymin=0, ymax=1.2)
plt.scatter(x=x, y=strong_eff_amd, s=200)
plt.scatter(x=x, y=strong_eff_int, s=200)
plt.legend(['AMD EPYC 7B12','Intel Xenon E5-2697'])

In [ ]:
#@title weak scaling efficiency plot
yamd = [
1.841,
2.027,
2.588,
2.800,
2.529,
2.718,
3.289,
3.370,
3.396,
4.025,
3.844,
3.850,
4.740,
4.590,
5.083,
4.947]

yint = [
1.796,
1.885,
1.967,
2.059,
2.131,
2.228,
2.310,
2.405,
3.187,
3.323,
3.420,
3.522,
3.653,
3.741,
3.820,
3.925]

nn = [
300000,
599798,
899596,
1199394,
1499192,
1798990,
2098788,
2398586,
2698384,
2998182,
3297980,
3597778,
3897576,
4197374,
4497172,
4796970]

weak_eff_amd = []
weak_eff_int = []
for i,_ in enumerate(yamd):
    weak_eff_amd.append(yamd[0]/yamd[i])
    weak_eff_int.append(yint[0]/yint[i])

plt.figure(figsize=(20,12))
plt.xlabel('thread', fontsize=18)
plt.ylabel('W(p)', fontsize=18)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.ylim(ymin=0, ymax=1.2)
plt.scatter(x=x, y=weak_eff_amd, s=200)
plt.scatter(x=x, y=weak_eff_int, s=200)

plt.legend(['AMD EPYC 7B12','Intel Xenon E5-2697'])

axes1 = plt.gca()
axes2 = axes1.twiny()
axes2.grid(False)
axes2.ticklabel_format(style='plain')
axes2.set_xticks(nn, True)
axes2.tick_params(axis='both', which='major', labelsize=16)
axes2.set_xlabel("N",{'size':18})


In [ ]:
#@title throughput plot
import numpy as np
import matplotlib.ticker as ticker

yv100 = np.array([
87196896.217,
208566721.581,
385276679.841,
769508037.018,
1477602230.483,
2967348837.209,
4743801039.346,
7798719512.195,
10931183255.019,
14059296896.457]) / 1e6

yp100 = np.array([
89363817.097,
188299454.635,
383382497.541,
766893203.883,
1456619331.195,
2354169741.697,
3398882978.723,
5011716300.940,
6970825388.177,
8314773004.141]) / 1e6

ymx130 = np.array([
86650602.409,
160591966.173,
249695805.315,
226592082.616,
451932916.429,
521481118.195,
586390749.747,
618721428.053,
649382716.049]) / 1e6

yt4 = np.array([
109567336.989,
235461872.287,
469476219.145,
930388692.579,
1250412898.151,
1844939271.255,
2552386658.677,
3605835917.676,
4535607940.446,
5775685433.826]) /1e6

nn = [
2000,
4000,
8000,
16000,
32000,
64000,
128000,
256000,
512000,
1024000]

nnlbl = [
'2K',
'4K',
'8K',
'16K',
'32K',
'64K',
'128K',
'256K',
'512K',
'1024K']

plt.figure(figsize=(20,12))
ax = plt.gca()
plt.xlabel('N', fontsize=18)
plt.ylabel('millions elements / s', fontsize=18)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.ylim(ymin=0, ymax=20000)
plt.scatter(x=nn, y=yv100, s=200, marker='o')
plt.scatter(x=nn, y=yp100, s=200, marker='o')
plt.scatter(x=nn, y=yt4, s=200, marker='o')
plt.scatter(x=nn[:-1], y=ymx130, s=200, marker='o')
plt.legend(['Tesla V100-SXM2','Tesla P100-PCIE','Tesla T4', 'GeForce MX130'])
ax.set_xscale("log")
ax.set_xticks(nn)
ax.get_xaxis().set_major_formatter(ticker.ScalarFormatter())
a=ax.get_xticks().tolist()
ax.set_xticklabels(a)
